In [1]:
from tinybig.config import rpn_config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'sst2_configs'
config_obj = rpn_config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.sst2', 'data_parameters': {'name': 'sst2', 'train_batch_size': 64, 'test_batch_size': 64}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 12, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.0001, 'weight_decay': 1e-05}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.35}}, 'loss_configs': {'loss_class': 'torch.nn.CrossEntropyLoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [1, 1, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_layer', 'm': 10325, '

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [3]:
print('instantiating objects from config...')
object_dict = config_obj.instantiate_object_from_config(config['configurations'])
data_obj, model_obj, learner_obj, metric_obj, result_obj = [object_dict[name] for name in ['data', 'model', 'learner', 'metric', 'result']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  53235


In [4]:
print('loading dataset...')
data_loader = data_obj.load()

loading dataset...


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...


100%|██████████| 1053/1053 [00:10<00:00, 98.75it/s, epoch=0/12, loss=0.277, lr=0.0001, metric_score=0.905, time=10.7] 


Epoch: 0, Test Loss: 0.49072653480938505, Test Score: 0.7947247706422018, Time Cost: 0.1188809871673584


100%|██████████| 1053/1053 [00:10<00:00, 102.24it/s, epoch=1/12, loss=0.243, lr=3.5e-5, metric_score=0.905, time=21.1]


Epoch: 1, Test Loss: 0.5090811933789935, Test Score: 0.7970183486238532, Time Cost: 0.07034993171691895


...

Epoch: 9, Test Loss: 0.5326536872557232, Test Score: 0.8061926605504587, Time Cost: 0.07764220237731934


100%|██████████| 1053/1053 [00:10<00:00, 98.53it/s, epoch=10/12, loss=0.221, lr=2.76e-9, metric_score=0.952, time=117]


Epoch: 10, Test Loss: 0.5276471003890038, Test Score: 0.8061926605504587, Time Cost: 0.0757441520690918


100%|██████████| 1053/1053 [00:10<00:00, 98.42it/s, epoch=11/12, loss=0.227, lr=9.65e-10, metric_score=0.857, time=127]


Epoch: 11, Test Loss: 0.528856194445065, Test Score: 0.8061926605504587, Time Cost: 0.07407307624816895
model checkpoint saving to ./ckpt/sst2_configs_checkpoint...


In [6]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [7]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
accuracy 0.8061926605504587


In [8]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
